In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

kospi200 = pd.read_csv("./data/data.csv", encoding="cp949")
kospi200 = kospi200[kospi200.columns[:2]]
kospi200["종목코드"] = kospi200["종목코드"].map('{:06d}'.format)

name_dict = { i : j for i, j in zip(kospi200["종목명"], kospi200["종목코드"]) }
code_dict = { j : i for i, j in zip(kospi200["종목명"], kospi200["종목코드"]) }

import pickle
with open('raw_data_20180824.pickle','rb') as handle:
    raw_data = pickle.load(handle)
    
copy_keys = kospi200["종목코드"]

In [24]:
for code in raw_data.keys():
    """
    X
    """
    raw_data[code]["pct_change"] = raw_data[code].Close.pct_change()
    for i in range(3, 31):
        raw_data[code]["close_ma_{}".format(i)] = raw_data[code].Close.rolling(window = i).mean()
        raw_data[code]["volume_ma_{}".format(i)] =  raw_data[code].Volume.rolling(window = i).mean()
    for i in [7, 21, 30, 60, 90, 91, 100, 120]:
        raw_data[code]["his_vol_{}".format(i)] = raw_data[code]["pct_change"].rolling(window = i).std()*(252**0.5)
    """
    Y
    """
    
    raw_data[code] = raw_data[code].assign(Y1 = raw_data[code].Close - raw_data[code].close_ma_7.shift(1))
    raw_data[code] = raw_data[code].assign(Y2 =
                                           raw_data[code]["pct_change"] - 
                                           raw_data[code]["pct_change"].rolling(window = 7).mean().shift(1))
    raw_data[code] = raw_data[code].assign(Y3 = raw_data[code].Close.rolling(window=7).mean() - 
                                           raw_data[code].Close.shift(1).rolling(window=7).mean())
    
    raw_data[code] = raw_data[code].assign(Y1 = np.where(raw_data[code].Y1>=0, 1, 0))
    raw_data[code] = raw_data[code].assign(Y2 = np.where(raw_data[code].Y2>=0, 1, 0))
    raw_data[code] = raw_data[code].assign(Y3 = np.where(raw_data[code].Y3>=0, 1, 0))
    """
    Drop NaN
    """
    raw_data[code] = raw_data[code].dropna(axis=0)

In [25]:
del_li2 = ['BGF', 'HDC', 'SK디스커버리', '대우조선해양', '대한항공', '동아쏘시오홀딩스', '동양', '롯데지주', '오리온홀딩스', '우리은행', '쿠쿠홀딩스', '팬오션', '한국타이어월드와이드', '한라홀딩스', '현대중공업', '효성'] 

In [26]:
del_li = [key for key, df in raw_data.items() if len(df) < 252*2]

In [27]:
len(raw_data)

202

In [28]:
for code1 in del_li:
    del raw_data[code1]
for code2 in del_li2:
    del raw_data[name_dict[code2]]

In [29]:
len(raw_data)

177

In [30]:
raw_data['005930'].head()

,Open,High,Low,Close,Adj Close,Volume,pct_change,close_ma_3,volume_ma_3,close_ma_4,...,his_vol_21,his_vol_30,his_vol_60,his_vol_90,his_vol_91,his_vol_100,his_vol_120,Y1,Y2,Y3
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-20,6940.0,7180.0,6830.0,7140.0,0.107662,59790000,0.065672,6860.000000,5.993500e+07,6895.0,...,0.839377,0.751253,0.799395,0.781216,0.779456,0.757865,0.727845,1,1,1
2000-06-21,7300.0,7380.0,7070.0,7080.0,0.106758,37585000,-0.008403,6973.333333,5.031167e+07,6915.0,...,0.822717,0.744535,0.778203,0.778126,0.777170,0.757627,0.716305,1,0,0
2000-06-22,7240.0,7270.0,6960.0,7040.0,0.106154,33635000,-0.005650,7086.666667,4.367000e+07,6990.0,...,0.814489,0.745024,0.760492,0.778106,0.773995,0.756560,0.716387,0,0,0
2000-06-23,6980.0,7010.0,6860.0,7000.0,0.105551,29215000,-0.005682,7040.000000,3.347833e+07,7065.0,...,0.803354,0.738580,0.753267,0.775556,0.773967,0.756696,0.716062,0,0,0
2000-06-26,7100.0,7360.0,6990.0,7210.0,0.108718,39980000,0.030000,7083.333333,3.427667e+07,7082.5,...,0.757796,0.732612,0.745944,0.772504,0.772424,0.757313,0.714942,1,1,1


In [84]:
import pickle

with open('raw_data_20180918.pickle', 'wb') as handle:
    pickle.dump(raw_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
def shift_dataset(raw_data, shift):
    data = {}
    for code in raw_data.keys():
        data[code] = pd.DataFrame()

        """
        Origin
        """
        data[code]["sh{}_Close".format(shift)] = raw_data[code]['Close'].shift(shift)
        data[code]["sh{}_Open".format(shift)] = raw_data[code]['Open'].shift(shift)
        data[code]["sh{}_High".format(shift)] = raw_data[code]['High'].shift(shift)
        data[code]["sh{}_Low".format(shift)] = raw_data[code]['Low'].shift(shift)
        data[code]["sh{}_Volume".format(shift)] = raw_data[code]['Volume'].shift(shift)
        data[code]["sh{}_Adj_Close".format(shift)] = raw_data[code]['Adj Close'].shift(shift)

        """
        X
        """
        data[code]["sh{}_pct_change".format(shift)] = raw_data[code]['pct_change'].shift(shift)
        for i in range(3, 31):
            data[code]["sh{}_close_ma_{}".format(shift, i)] = raw_data[code]["close_ma_{}".format(i)]
            data[code]["sh{}_volume_ma_{}".format(shift, i)] =  raw_data[code]["volume_ma_{}".format(i)]
        for i in [7, 21, 30, 60, 90, 91, 100, 120]:
            data[code]["sh{}_his_vol_{}".format(shift, i)] = raw_data[code]["his_vol_{}".format(i)]

        """
        Y
        """
        data[code]['Y1'] = raw_data[code]['Y1']
        data[code]['Y2'] = raw_data[code]['Y2']
        data[code]['Y3'] = raw_data[code]['Y3']

        """
        Drop NaN
        """
        data[code] = data[code].dropna(axis=0)
    return data

In [39]:
data_sh5 = shift_dataset(raw_data, shift=5)
data_sh1 = shift_dataset(raw_data, shift=1)

In [50]:
def adjust_window(raw_data, start, end):
    data = {}
    for code in raw_data.keys():
        data[code] = raw_data[code].loc[start:end].copy()
        data[code] = data[code].dropna(axis=0)
    return data

In [51]:
data5 = adjust_window(data_sh5, '2012-01-01', '2018-07-31')
data1 = adjust_window(data_sh1, '2012-01-01', '2018-07-31')

In [53]:
data5['005930'].head()

,sh5_Close,sh5_Open,sh5_High,sh5_Low,sh5_Volume,sh5_Adj_Close,sh5_pct_change,sh5_close_ma_3,sh5_volume_ma_3,sh5_close_ma_4,...,sh5_his_vol_21,sh5_his_vol_30,sh5_his_vol_60,sh5_his_vol_90,sh5_his_vol_91,sh5_his_vol_100,sh5_his_vol_120,Y1,Y2,Y3
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,21360.0,21120.0,21480.0,21120.0,12299050.0,16.105038,0.015209,21326.666667,1.169970e+07,21360.0,...,0.337341,0.379040,0.353275,0.370516,0.374890,0.399323,0.388828,1,1,1
2012-01-03,21320.0,21340.0,21500.0,21320.0,6216250.0,16.074879,-0.001873,21620.000000,1.366767e+07,21520.0,...,0.335360,0.379134,0.354569,0.369246,0.369723,0.391930,0.388958,1,1,1
2012-01-04,21460.0,21360.0,21600.0,21060.0,11133650.0,16.180435,0.006567,21766.666667,1.572760e+07,21615.0,...,0.343009,0.386705,0.358683,0.370770,0.370102,0.393982,0.390354,1,0,1
2012-01-05,21220.0,21460.0,21460.0,21200.0,11023850.0,20.859589,-0.011184,21600.000000,1.710995e+07,21600.0,...,0.345094,0.382645,0.361313,0.373654,0.371607,0.394996,0.391369,0,0,0
2012-01-06,21160.0,21240.0,21540.0,21140.0,10923900.0,20.800606,-0.002828,21166.666667,1.773945e+07,21400.0,...,0.346474,0.386126,0.362219,0.374843,0.372866,0.395989,0.392097,0,0,0


In [68]:
def modelfit(alg, dtrain, predictors, target, useTrainCV = True, 
             cv_folds = 5, early_stopping_rounds = 50, top = 10, 
             figure = True, model_report = True):
       
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_stdv=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target], eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    if model_report:
        #Print model report:
        print("Model Report")
        print("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
        print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
                    
    imp_fear_name = pd.Series(predictors)[alg.feature_importances_ > 0.01]
    imp_fear_name.index = range(len(imp_fear_name))
    imp_fear = pd.concat((imp_fear_name, 
               pd.Series(alg.feature_importances_[alg.feature_importances_ > 0.01])),axis=1).sort_values(by=1,ascending=True)
    imp_fear.index = range(len(imp_fear))
    imp_fear.columns = ["feature_name", "feature_importance"]
#     {i:j for i,j in zip(imp_fear.index, imp_fear["feature_name"])}
    if figure:
        if top:
            imp_fear.iloc[-top:].plot(y='feature_importance', x='feature_name', kind='barh', legend=False, figsize=(8,5))
        else:
            imp_fear.plot(y='feature_importance', x='feature_name', kind='barh', legend=False, figsize=(8,5))
    
#     feat_imp = pd.Series(alg.feature_importances_).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances', figsize=(200,5))
#     plt.ylabel('Feature Importance Score')
    
    return alg, imp_fear

In [55]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb_clf = XGBClassifier(random_state=42)

In [57]:
alg = XGBClassifier(
 learning_rate =0.15,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

predictors1 = [x for x in data1['005930'].columns if x not in ['Y1','Y2','Y3']]
predictors5 = [x for x in data5['005930'].columns if x not in ['Y1','Y2','Y3']]
target = "Y3"

In [59]:
def train_test_split(df, train_ratio, X, y, random = False, dtrain = False):
    """
    Argument
    df : DataFrame object
    train_ratio : float, in range(0,1)
    X : sequence data, such as list, tuple (Train Features)
    y : str
    random : boolean, you need "numpy" library cause we use np.random.permutation function.
    
    Return
    train : 학습시킬 Feature data
    train_idx : 지도학습의 Y value
    test : test 검증할 Feature data
    test_idx : test set의 Y value
    """
    train_size = int(len(df)*train_ratio)
    if random:
        shuffle_indicies = np.random.permutation(len(df))
        train_indicies = shuffle_indicies[:train_size]
        test_indicies = shuffle_indicies[train_size:]
    else:
        normal_indicies = np.arange(len(df))
        train_indicies = normal_indicies[:train_size]
        test_indicies = normal_indicies[train_size:]
    if dtrain:
        train = df.iloc[:train_size]
        train_idx = df.iloc[:train_size]
        test = df.iloc[train_size:]
        test_idx = df.iloc[train_size:]
    else:
        train = df.iloc[:train_size][X]
        train_idx = df.iloc[:train_size][y]
        test = df.iloc[train_size:][X]
        test_idx = df.iloc[train_size:][y]
    return train, train_idx, test, test_idx

In [60]:
train1 = {}
test1 = {}
train5 = {}
test5 = {}
for code, df in data1.items():
    X_train, y_train, X_test, y_test = train_test_split(df, 0.8, predictors1, target, dtrain=True)
    train1[code] = X_train
    test1[code] = X_test
for code, df in data5.items():
    X_train, y_train, X_test, y_test = train_test_split(df, 0.8, predictors5, target, dtrain=True)
    train5[code] = X_train
    test5[code] = X_test

In [69]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

def temp_calc_feature_importance(alg, train, test, predictors, target):
    test_result = {}
    fi_dict = {}
    for i in train.keys():
        model, fear_importance = modelfit(alg, train[i], predictors, target, 
                                                  cv_folds=5, early_stopping_rounds=20,
                                                  figure=False, model_report=False)
        pred = model.predict(test[i][predictors])
        # Calculating Score
        test_result[i] = np.array([
            metrics.accuracy_score(pred, test[i][target]),
            metrics.f1_score(pred, test[i][target]),
            metrics.precision_score(pred, test[i][target]),
            metrics.recall_score(pred, test[i][target]),
            metrics.roc_auc_score(pred, test[i][target])
        ])
        fi_dict[i] = {feature:value for feature,value in fear_importance.iloc[-10:].values}
    return test_result, fi_dict

In [66]:
import time
from sklearn import metrics

In [70]:
result1, fi_dict1 = temp_calc_feature_importance(alg, train1, test1, predictors1, target)
result5, fi_dict5 = temp_calc_feature_importance(alg, train5, test5, predictors5, target)

In [73]:
test_result1 = pd.DataFrame(result1, index=['accuracy_score', 'f1_score', 'precision_score', 'recall_score', 'roc_auc_score'])
test_result5 = pd.DataFrame(result5, index=['accuracy_score', 'f1_score', 'precision_score', 'recall_score', 'roc_auc_score'])

In [76]:
test_result1.T.describe()

,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score
count,177.000000,177.000000,177.000000,177.000000,177.000000
mean,0.637570,0.624001,0.641554,0.642941,0.650616
std,0.064416,0.103086,0.166397,0.105292,0.063330
min,0.398148,0.237288,0.166667,0.236842,0.407434
25%,0.591362,0.573964,0.537500,0.584416,0.609352
50%,0.638889,0.643963,0.662791,0.645833,0.656250
75%,0.679012,0.694215,0.760000,0.713178,0.694970
max,0.783951,0.809783,0.961039,0.859504,0.784789


In [77]:
test_result5.T.describe()

,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score
count,177.000000,177.000000,177.000000,177.000000,177.000000
mean,0.640523,0.610913,0.606779,0.666351,0.659310
std,0.072878,0.112886,0.178281,0.118536,0.068878
min,0.413580,0.172973,0.101266,0.225806,0.423250
25%,0.589506,0.549020,0.485030,0.592593,0.614880
50%,0.635802,0.627119,0.615894,0.678571,0.660127
75%,0.694444,0.692537,0.745283,0.758621,0.710754
max,0.817901,0.836565,0.967532,0.934211,0.817469


In [80]:
summation = {}
for feature in predictors1:
    summation[feature] = 0
for feature_importance in fi_dict1.values():
    for feature in feature_importance.keys():
        summation[feature] += feature_importance[feature]

In [81]:
summation2 = {}
for feature in predictors5:
    summation2[feature] = 0
for feature_importance in fi_dict5.values():
    for feature in feature_importance.keys():
        summation2[feature] += feature_importance[feature]

In [82]:
pd.Series(summation).sort_values(ascending=False).head(10)

sh1_close_ma_3     14.261811
sh1_his_vol_7       9.855589
sh1_pct_change      8.044199
sh1_his_vol_21      5.337510
sh1_Volume          5.148248
sh1_his_vol_60      5.018664
sh1_his_vol_30      4.447597
sh1_his_vol_120     3.600136
sh1_his_vol_100     2.325098
sh1_volume_ma_3     2.178839
dtype: float64

In [83]:
pd.Series(summation2).sort_values(ascending=False).head(10)

sh5_close_ma_3     22.426389
sh5_his_vol_7       9.521069
sh5_pct_change      9.220807
sh5_his_vol_21      5.001822
sh5_his_vol_60      4.644510
sh5_his_vol_30      4.199361
sh5_Volume          3.533250
sh5_his_vol_120     3.471783
sh5_Open            3.450742
sh5_his_vol_90      2.246669
dtype: float64